In [35]:
import pandas as pd
import numpy as np
import re
import pymysql
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

In [29]:
def load_data(columns, table):
    conn = pymysql.connect(port = 3306,
                           user = 'root',
                           passwd = 'gusrl2600-',
                           host = '127.0.0.1',
                           db = 'product_db',
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cur:

            query = "SELECT " + columns + " FROM " + table + ";"
                
            cur.execute(query)
            data = cur.fetchall()
            
    finally:
        conn.close()
        
    return data

In [14]:
review = pd.DataFrame(load_data('*','review_info'))
product = pd.DataFrame(load_data('*','product_info'))

In [30]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    120 non-null    object
 1   title   120 non-null    object
 2   price   120 non-null    int64 
 3   rating  120 non-null    object
 4   volunm  120 non-null    int64 
 5   imgurl  120 non-null    object
dtypes: int64(2), object(4)
memory usage: 5.8+ KB


In [17]:
# 코드 최대길이
print('code 최대길이 : ', max([len(i) for i in product['code']]))

# 코드 유니크한 값
print('code 유니크한 값 :', product['code'].nunique())

# title 최대길이
print('title 최대길이 : ', max([len(i) for i in product['title']]))

# title 유니크한 값
print('title 유니크한 값 : ' ,product['title'].nunique())

# price 최대값
print('price 최대값 : ', max(product['price']))

# volumn 최대값
print('volunm 최대값 : ', max(product['volunm']))

# imgurl 최대길이
print('imgurl 최대길이 : ', max([len(i) for i in product['imgurl']]))

code 최대길이 :  10
code 유니크한 값 : 120
title 최대길이 :  157
title 유니크한 값 :  112
price 최대값 :  32767
volunm 최대값 :  16195
imgurl 최대길이 :  89


In [18]:
review.head()

,item_id,review_id,login_id,score,content,item_option,date,likeCnt
0,24181889,100016281,ross****,5.0,배송빠르고 잘작동합니다. 많이파세요,[옵션]제품선택:소니 DR-BT101,2012-02-24,0
1,228322167,100019388,hwbs****,5.0,일단 가볍고 간단해서 좋구요.... 저희 신랑이랑 커플로 2개 샀네요.,[옵션]제품선택:아이리버 BT-S10 블루,2012-02-24,0
2,24181889,100023537,peel**,5.0,배송 빠르고 좋네요~,[옵션]제품선택:소니 DR-BT101,2012-02-24,0
3,24181889,100028785,moon*****,5.0,만족합니다,[옵션]제품선택:소니 DR-BT101,2012-02-24,0
4,60614626,100060926,tore****,5.0,만족합니다,[옵션]색상선택:소니 DRC-BT30 블랙,2012-02-25,0


In [19]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60582 entries, 0 to 60581
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_id      60582 non-null  object
 1   review_id    60582 non-null  object
 2   login_id     60582 non-null  object
 3   score        60582 non-null  object
 4   content      60582 non-null  object
 5   item_option  60582 non-null  object
 6   date         60582 non-null  object
 7   likeCnt      60582 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 3.7+ MB


In [27]:
print('item_id 최대길이 : ', max([len(i) for i in review['item_id']]))
print('review_id 최대길이 : ', max([len(i) for i in review['review_id']]))
print('login_id 최대길이 : ', max([len(i) for i in review['login_id']]))
print('content 최대길이 : ', max([len(i) for i in review['content']]))
print('item_option 최대길이 : ', max([len(i) for i in review['item_option']]))
print('date 최대길이 : ', max([len(i) for i in review['date']]))
print('likeCnt 최대값', max(review['likeCnt']))

item_id 최대길이 :  10
review_id 최대길이 :  9
login_id 최대길이 :  31
content 최대길이 :  2531
item_option 최대길이 :  98
date 최대길이 :  11
likeCnt 최대값 2015


# image 저장

In [71]:
product['title']

0                         비트플러스 X880 블루투스 헤드셋/무선/헤드폰/이어폰
1                        아이리버 BT-S30 블루투스 이어폰/무선 헤드폰,헤드셋
2                        아이리버 BEP-L20 블루투스 이어폰/무선헤드셋,헤드폰
3      고감도 블루투스 이어폰 고감도 스테레오/음성안내/이어셋/스마트폰 통화+음악감상/ 자...
4      국산 블루스톰/바이브스톰 스테레오 블루투스 이어폰/헤드셋 BS450/BS-450/무...
                             ...                        
115    모본 정품 MF230 블루투스 헤드셋/모노&스테레오 겸용/통화+음악감상/멀티페어링/...
116                       소니 WI-C300 블루투스 이어폰/넥밴드/무선 헤드셋
117    아이테크 정품 VoiceClip 604 블루투스 헤드셋/멀티포인트,휴대폰2대 동시연...
118    뷰젯 Buzzet 초소형 초경량 무선핸즈프리 커널형 헤드셋 블루투스 연속통화 4시간...
119    요즘대세 브리츠 BE-M16 무선이어폰/통화가능/고음질/초경량/이어훅제공/파우치/블루투스
Name: title, Length: 120, dtype: object

In [33]:
import urllib.request

In [40]:
for i in product['imgurl']:
    print(i)

https://cdn.011st.com/11dims/resize/160/11src/pd/17/8/5/5/7/5/0/1000855750_L300_V1.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/17/1/9/0/7/5/9/10190759_B_V1.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/17/5/2/7/4/1/0/dLQYG/1033527410_L300.jpg
https://cdn.011st.com/11dims/resize/160/11src/ak/0/4/1/6/9/6/1049041696_L300_V1.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/20/1/4/5/5/3/8/NOVRI/105145538_B.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/19/3/3/7/0/5/9/vBRHk/1065337059_L300.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/17/3/2/8/4/2/7/1079328427_L300.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/20/2/7/8/2/3/3/epRBi/1081278233_L300.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/20/1/9/4/7/9/6/CHHUh/108194796_B.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/17/1/0/0/2/1/8/1087100218_B_V1.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/17/1/0/0/0/2/2/1092100022_B_V1.jpg
https://cdn.011st.com/11dims/resize/160/11src/pd/17/5

In [85]:
for i in range(len(product)):
    urllib.request.urlretrieve(product['imgurl'][i],'./venv/img/img'+str(i)+'.'+product['imgurl'][0][-3:])
    print('img'+str(i)+'.'+product['imgurl'][0][-3:])

img0.jpg
img1.jpg
img2.jpg
img3.jpg
img4.jpg
img5.jpg
img6.jpg
img7.jpg
img8.jpg
img9.jpg
img10.jpg
img11.jpg
img12.jpg
img13.jpg
img14.jpg
img15.jpg
img16.jpg
img17.jpg
img18.jpg
img19.jpg
img20.jpg
img21.jpg
img22.jpg
img23.jpg
img24.jpg
img25.jpg
img26.jpg
img27.jpg
img28.jpg
img29.jpg
img30.jpg
img31.jpg
img32.jpg
img33.jpg
img34.jpg
img35.jpg
img36.jpg
img37.jpg
img38.jpg
img39.jpg
img40.jpg
img41.jpg
img42.jpg
img43.jpg
img44.jpg
img45.jpg
img46.jpg
img47.jpg
img48.jpg
img49.jpg
img50.jpg
img51.jpg
img52.jpg
img53.jpg
img54.jpg
img55.jpg
img56.jpg
img57.jpg
img58.jpg
img59.jpg
img60.jpg
img61.jpg
img62.jpg
img63.jpg
img64.jpg
img65.jpg
img66.jpg
img67.jpg
img68.jpg
img69.jpg
img70.jpg
img71.jpg
img72.jpg
img73.jpg
img74.jpg
img75.jpg
img76.jpg
img77.jpg
img78.jpg
img79.jpg
img80.jpg
img81.jpg
img82.jpg
img83.jpg
img84.jpg
img85.jpg
img86.jpg
img87.jpg
img88.jpg
img89.jpg
img90.jpg
img91.jpg
img92.jpg
img93.jpg
img94.jpg
img95.jpg
img96.jpg
img97.jpg
img98.jpg
img99.jpg
img100.jpg

In [80]:
image = pd.DataFrame(product['title'])

In [81]:
image['X'] = 1

In [82]:
image['Y'] = 1

In [83]:
image.head()

,title,X,Y
0,비트플러스 X880 블루투스 헤드셋/무선/헤드폰/이어폰,1,1
1,"아이리버 BT-S30 블루투스 이어폰/무선 헤드폰,헤드셋",1,1
2,"아이리버 BEP-L20 블루투스 이어폰/무선헤드셋,헤드폰",1,1
3,고감도 블루투스 이어폰 고감도 스테레오/음성안내/이어셋/스마트폰 통화+음악감상/ 자...,1,1
4,국산 블루스톰/바이브스톰 스테레오 블루투스 이어폰/헤드셋 BS450/BS-450/무...,1,1


In [86]:
image.to_csv('./venv/image.csv',index=False,sep='|',encoding='utf-8')